In [ ]:
import os
import pandas as pd
import numpy as np
import math
import scipy as sp
import scipy.signal as sg
import scipy.fftpack
cwd="/Users/ashashibu/Desktop/WaterYears" #Directory
os.chdir(cwd)
df=pd.read_excel('2004.xlsx')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['DATE']= df['Date']
df = df.set_index('Date')
df=df.loc[df.index.notnull()]
df = df.sort_index()

#Headwater Smoothing Technique
df.HeadWater=df.HeadWater.where(df.HeadWater.between(960,1034)) #Setting min and max elevation levels
df['HeadWater'] = df['HeadWater'].interpolate(method='linear', axis=0).bfill() #replacing 0 values of headwater
df=df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
HW=df['HeadWater']
date = df.index
b, a = sg.butter(2, 1/300) #Second order butterworth filter
df['HeadWater']=sg.filtfilt(b, a, HW)

#Computing storage change and caluculating initial inflow
df['Reservoir Volume']=(5772603.82*(df['HeadWater']*df['HeadWater']))-(10401155398.61*df['HeadWater'])+469216158531776
df['Change in Storage'] = df['Reservoir Volume'].diff()
df['Inflow(C)']=(df['Change in Storage']/300)+df['PlantTotalFlow']
df=df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

#Replacing negative inflow terms and adjusting with water years
Pos= df[df['Inflow(C)'] > 0] #Positive Inflows
Neg= df[df['Inflow(C)']< 0]  #Negative Inflows
R=(abs(Neg['Inflow(C)'].sum()))/(Pos['Inflow(C)'].sum()) 
df[df['Inflow(C)']<0]=0
df['Inflow']=df['Inflow(C)']-(df['Inflow(C)']*R)
df.drop('Inflow(C)', axis=1, inplace=True)
df.reset_index(drop=False, inplace=False)
df = df.loc[df['HeadWater'] != 0]
df.to_csv('2004.csv',index = True)

